<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/4_LIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split

Mounted at /content/drive


In [2]:
import sys 
sys.path.append('/content/drive/MyDrive/Tesi_magistrale/Tesi_magistrale')
import mf

# Dataset import

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
data = mf.feature_scaling(data)
data = mf.feature_engineering(data)
data = pd.get_dummies(data)
feature_names = mf.get_col(data)

# Model import

### Catboost 

In [4]:
# data, categorical_names = mf.encoding(data, 'cat_sign_col')
# categorical_features = np.arange(176,206)

# X_train = mf.get_set('train_id', data, 'catboost_new')
# X_val = mf.get_set('val_id', data, 'catboost_new')
# X_test = mf.get_set('test_id', data, 'catboost_new')

# X_train = mf.to_list(X_train, categorical_features)
# X_val = mf.to_list(X_val, categorical_features)
# X_test = mf.to_list(X_test, categorical_features)

Riferimento: https://stackoverflow.com/questions/30483246/how-to-check-if-a-python-module-has-been-imported

In [5]:
# load_catboost = False
# if load_catboost:
#   if 'catboost' not in sys.modules:
#     !pip install catboost 
#   from catboost import CatBoostClassifier
#   cat = CatBoostClassifier()
#   cat.load_model('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/modelli/catboost_new')  

## XGBoost

In [6]:
categorical_features = np.arange(178,1091).tolist()

In [7]:
X_train = mf.get_set('train_id', data, 'xgboost')
X_val = mf.get_set('val_id', data, 'xgboost')
X_test = mf.get_set('test_id', data, 'xgboost')

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
load = True
if load:
  from joblib import load
  from xgboost import XGBClassifier

  xgb = load("/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/xgboost/xgb.joblib.dat")

# LIME

**LIME Local interpretable model-agnostic explanations**

1. LIME generates a new dataset consisting of: 
  * permuted samples, drawing from a normal distribution with mean and standard deviation taken from the feature (in case of tabular data);
  * the corresponding predictions of the black box model;
2. On this new dataset LIME then trains an interpretable model;
3. The model is weighted by the proximity of the sampled instances to the instance of interest.

The **recipe for training local surrogate models**:
1. Select your instance of interest for which you want to have an explanation of its black box prediction.
2. Perturb your dataset and get the black box predictions for these new points.
3. Weight the new samples according to their proximity to the instance of interest.
4. Train a weighted, interpretable model on the dataset with the variations.
5. Explain the prediction by interpreting the local model.

Tutorial LIME: https://marcotcr.github.io/lime/tutorials/Tutorial%20-%20continuous%20and%20categorical%20features.html

Documentazione LIME: https://lime-ml.readthedocs.io/en/latest/lime.html

Repository github di lime: https://github.com/marcotcr/lime

In [9]:
!pip install lime

     |████████████████████████████████| 276kB 12.0MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp36-none-any.whl size=283846 sha256=13ec27c3fd7657fae15057d051d8456ef259084a617c9a5ff688e4f5fdb6e5a8
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime


Riferimento: https://stackoverflow.com/questions/13021654/get-column-index-from-column-name-in-python-pandas

Riferimento: https://stackoverflow.com/questions/52404971/get-a-list-of-categories-of-categorical-variable-python-pandas

In [26]:
def lime(X_train, X_test, categorical_features, feature_names, clf, n_features, j):
  import lime
  import lime.lime_tabular

  # setup dell'esxplainer
  explainer = lime.lime_tabular.LimeTabularExplainer(X_train.to_numpy(), 
                                                      feature_names=feature_names,
                                                      class_names=[0,1],
                                                      categorical_features=categorical_features,
                                                      categorical_names=None,
                                                      verbose=True, mode='classification')#,
                                                      # discretize_continuous = True, discretizer='decile')
  
  # explanation
  exp = explainer.explain_instance(X_test.to_numpy()[j], 
                                   clf.predict_proba, num_features=n_features)
  exp.show_in_notebook(show_table=True)
  return

In [27]:
lime(X_train[:10000], X_test, categorical_features, feature_names, xgb, 10, 2)

ValueError: ignored